In [1]:
from ultralytics import YOLO
import os  
import pandas as pd  
import numpy as np  
import cv2  
import matplotlib.pyplot as plt  
import seaborn as sns  
from sklearn.model_selection import train_test_split 
from glob import glob  
 
from PIL import Image


import warnings
warnings.filterwarnings("ignore", "use_inf_as_na option is deprecated")

In [ ]:

# Create necessary directories
!mkdir -p "/content/kaggle/working/data"
!mkdir -p "/content/kaggle/working/data/images"
!mkdir -p "/content/kaggle/working/data/images/train"
!mkdir -p "/content/kaggle/working/data/images/val"
!mkdir -p "/content/kaggle/working/data/labels"
!mkdir -p "/content/kaggle/working/data/labels/train"
!mkdir -p "/content/kaggle/working/data/labels/val"

root_dir = "/content/kaggle/working/data"
labels_dir = "/content/kaggle/working/data/labels"
images_dir = "/content/kaggle/working/data/images"

A subdirectory or file D:\YOLOv8_car_detection/kaggle/working/data already exists.
Error occurred while processing: D:\YOLOv8_car_detection/kaggle/working/data.
A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file D:\YOLOv8_car_detection/kaggle/working/data/images already exists.
Error occurred while processing: D:\YOLOv8_car_detection/kaggle/working/data/images.
A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file D:\YOLOv8_car_detection/kaggle/working/data/images/train already exists.
Error occurred while processing: D:\YOLOv8_car_detection/kaggle/working/data/images/train.
A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file D:\YOLOv8_car_detection/kaggle/working/data/images/val already exists.
Error occurred while processing: D:\YOLOv8_car_detection/kaggle/working/data/images/val.
A subdirectory or file -p already exists.
Error occurred 

In [ ]:
train_data = "/content/data/training_data"
csv_data = "/content/data/train_solution_bounding_boxes (1).csv"
test_data = "/content/data/testing_data"

In [ ]:
df = pd.read_csv(csv_data)
df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [6]:
# Prepare YOLO format annotations
def create_yolo_annotation(row, img_width, img_height):
    x_center = ((row['xmin'] + row['xmax']) / 2) / img_width
    y_center = ((row['ymin'] + row['ymax']) / 2) / img_height
    width = (row['xmax'] - row['xmin']) / img_width
    height = (row['ymax'] - row['ymin']) / img_height
    return f"0 {x_center} {y_center} {width} {height}"

In [11]:
# Create YOLO annotations and copy images
for img_name in df['image'].unique():
    img_df = df[df['image'] == img_name]
    img_path = os.path.join(train_data, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img_height, img_width = img.shape[:2]
        
        # Decide whether to put in train or val folder
        if np.random.rand() < 0.8:  # 80% train, 20% val
            subset = "train"
        else:
            subset = "val"
        
        # Copy image
        dst_img_path = os.path.join(images_dir, subset, img_name)
        cv2.imwrite(dst_img_path, img)
        
        # Create annotation file
        annotation_path = os.path.join(labels_dir, subset, f"{img_name.split('.')[0]}.txt")
        with open(annotation_path, 'w') as f:
            for _, row in img_df.iterrows():
                yolo_annotation = create_yolo_annotation(row, img_width, img_height)
                f.write(yolo_annotation + '\n')

In [12]:
# Create YAML configuration file
yaml_content = f"""
path: {root_dir}
train: images/train
val: images/val

nc: 1
names: ['car']
"""

with open('car_detection.yaml', 'w') as f:
    f.write(yaml_content)

print("YAML configuration file created.")

YAML configuration file created.


In [13]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 44.2MB/s]


In [14]:
# Disable W&B logging to avoid the API key prompt
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
results = model.train(
    data='car_detection.yaml',
    epochs=30,
    imgsz=640,
    batch=16,
    name='car_detection_model',
    device='cuda'
)

In [ ]:
# Model Evaluation
results = model.val()
print(results)

In [ ]:
# Test on a sample image from the test set
sample_test_image = os.path.join(test_data, os.listdir(test_data)[0])

# Perform inference on the sample image
test_results = model(sample_test_image)

# Display the first result using 'plot' or similar method
test_results[0].plot()  # This should display the image with predictions